## Introduction
This notebook polls an agency feed to get the set of vehicle identifiers so that assets can be created in the AssetLibrary.

Currently there are quirks that require manually creating some entities  with the cdf manager (e.g. https://cdfmanager.developgtt.com/) so that supporting resources can be created by the backend.


Point `config_folder` to a folder with agency json config files, or set `config_files` manually.


In [ ]:
import json
from pathlib import Path

agency_name = "cdta"

config_folder = Path("../tsp_gtfs_realtime/config/agencies")
csv_folder = Path("./output/csv_vehicles/")
csv_folder.mkdir(parents=True, exist_ok=True)

# config_files = sorted(config_folder.glob("*.json"))
config_file = config_folder.joinpath(f"{agency_name}.json")
csv_file = csv_folder.joinpath(f"{agency_name}.csv")
with config_file.open(mode="r") as fp:
    agency_config = json.load(fp)


Poll each `vehicle_positions` feed `num_samples` times in order to get the set of vehicle identifiers (either id or label)

Instantiating the pollers outside the loop allows them each to manage the polling rate. This could be parallelized.


In [ ]:
from tsp_gtfs_realtime.core.gtfs_realtime import GTFSRealtimeAPIPoller, GTFSRealtimeConfig

gtfs_realtime_api_poller = GTFSRealtimeAPIPoller(
    GTFSRealtimeConfig.from_inputs(config_file=config_file)
)
# load previously saved vehicles if they exist
gtfs_realtime_vehicle_ids = (
    set(csv_file.open(mode="r").read().split("\n"))
    if csv_file.is_file()
    else set()
)

num_samples = 10
for sample_idx in range(num_samples):
    print(f"Sample {sample_idx}, Polling {agency_name}")
    try:
        gtfs_realtime_api_poller.poll_vehicle_positions()
        vehicle_ids = set(v for v, _ in gtfs_realtime_api_poller.vehicle_positions)
        gtfs_realtime_vehicle_ids.update(vehicle_ids)
    except Exception as e:
        print(e)
        print(f"Error, skipping {agency_name} {sample_idx}")
        continue


Save data to a csv file, so that it can be updated over time to get vehicles that were not active during previous polling

In [ ]:
should_save_csv = True

if should_save_csv:
    with csv_file.open(mode="w") as fp:
        print(
            *sorted(
                gtfs_realtime_vehicle_ids,
                key=int if all(v.isdigit() for v in gtfs_realtime_vehicle_ids) else None
            ),
            sep="\n",
            end="",
            file=fp
        )

## Create assets
Use sigv4 to post `region` and `agency` groups, and `vehicles` and `integration_com` devices

In [ ]:
from gtt.service.asset_library import AssetLibraryAPI

asset_library_api = AssetLibraryAPI("https://oo9fn5p38b.execute-api.us-east-1.amazonaws.com/Prod")

## Region
Create region with the same name as the agency

Note: this does not really work because the cdfmanager somehow handles the creation of the `caCertId` and `regionGUID`. I think that when API Gateway is queried, it calls a subsequent lambda, but the cdf manager calls that lambda itself. There is something wrong different with the way they do this, so the automatic creation does not work.

### cdfmanager should be used to create Region

In [ ]:
# from gtt.service.asset_library import Region
# from typing import Optional
# from pydantic import Field

# class RegionWriter(Region):
#     ca_cert_id: Optional[str] = Field(alias="caCertId")
#     unique_id: Optional[str] = Field(alias="regionGUID")  # note name change

#     def create(self):
#         data = {
#             "attributes": {
#                 "description": self.description,
#                 "caCertId": self.ca_cert_id,
#                 "regionGUID": self.unique_id,
#                 "displayName": self.display_name,
#             },
#             "category": self.category,
#             "templateId": self._template_id,
#             "name": self.name,
#             "groupPath": self.group_path,
#             "parentPath": self.parent_path,
#         }
#         groups_url = f"{asset_library_api.url}/groups"
#         asset_library_api._send_request(method="POST", url=groups_url, data=data)


In [ ]:
# for agency_name in agency_vehicles:
#     cdf_backend_region = RegionWriter(
#         **{
#             "attributes": {
#                 "caCertId": "NULL_CA",
#                 "regionGUID": "NULL_GUID",
#                 "displayName": agency_name,
#             },
#             "description": f"region for {agency_name}",
#             "name": agency_name,
#             "category": "group",
#             "templateId": "region",
#             "groupPath": f"/{agency_name.lower()}",
#             "parentPath": "/",
#         }
#     )
#     cdf_backend_region.create()

In [ ]:
region = asset_library_api.get_region(agency_name.lower())

## Agency
Create region with the same name as the agency

Note: this does not really work because the cdfmanager somehow handles the creation of the `agencyID`, `vpsCertId`, `Cert2100Id`, `CMSId`, and `caCertId`. I think that when API Gateway is queried, it calls a subsequent lambda, but the cdf manager calls that lambda itself. There is something wrong different with the way they do this, so the automatic creation does not work.

### cdfmanager should be used to create Agency

In [ ]:
# from gtt.service.asset_library import Agency
# from typing import Optional
# from pydantic import Field

# class AgencyWriter(Agency):
#     ca_cert_id: Optional[str] = Field(alias="caCertId")
#     unique_id: Optional[str] = Field(alias="regionGUID")  # note name change

#     def create(self):
#         data = {
#             "attributes": {
#                 "city": self.city,
#                 "state": self.state,
#                 "timezone": self.timezone,
#                 "agencyCode": self.agency_code,
#                 "agencyID": self.unique_id,
#                 "vpsCertId": self.vps_cert_id,
#                 "Cert2100Id": self.cert2100_id,
#                 "priority": self.priority,
#                 "CMSId": self.cms_id,
#                 "caCertId": self.ca_cert_id,
#                 "displayName": self.display_name,
#             },
#             "category": self.category,
#             "templateId": self._template_id,
#             "name": self.name,
#             "groupPath": self.group_path,
#             "parentPath": self.parent_path,
#         }
#         groups_url = f"{asset_library_api.url}/groups"
#         asset_library_api._send_request(method="POST", url=groups_url, data=data)


In [ ]:
# cdf_backend_agency = AgencyWriter(
#     **{
#         "attributes": {
#             "city": agency_config.get("city", "New York"),
#             "state": agency_config.get("state", "NY"),
#             "timezone": agency_config.get("timezone", "Eastern"),
#             "agencyCode": agency_config.get("agencyCode", 250),
#             "agencyID": agency_config.get("agencyID", "NULL_GUID"),
#             "vpsCertId": agency_config.get("vpsCertId", "NULL_CERT"),
#             "Cert2100Id": agency_config.get("Cert2100Id", "NULL_CERT"),
#             "priority": agency_config.get("priority", "Low"),
#             "CMSId": agency_config.get("CMSId", "NULL_GUID"),
#             "caCertId": agency_config.get("caCertId", "NULL_CA"),
#             "displayName": agency_config.get("displayName", agency_name),
#         },
#         "category": "group",
#         "templateId": "agency",
#         "name": agency_name,
#         "groupPath": f"/{agency_name.lower()}/{agency_name.lower()}",
#         "parentPath": f"/{agency_name.lower()}",
#         "description": agency_config.get("description", f"agency for {agency_name}"),
#     }
# )
# cdf_backend_agency.create()

In [ ]:
agency = asset_library_api.get_agency(agency_name, agency_name)

## Vehicle
Create Vehicles under the agency group

In [ ]:
from gtt.service.asset_library import Vehicle
from botocore.compat import quote
import json

class VehicleWriter(Vehicle):
    def create(self):
        data = {
            "attributes": {
                "VID": int(self.VID),
                "name": self.name,
                "priority": self.priority,
                "type": self.type_,
                "class": int(self.class_),
                "uniqueId": self.unique_id,
            },
            "description": self.description,
            "category": self.category,
            "templateId": self._template_id,
            "state": self.state,
            "deviceId": self.device_id,
            "groups": {
                "out": {"ownedby": [f"/{agency_name}/{agency_name}"]}
            },
            "devices": {
                "out": {"installedat": self.installed_device_ids}
            },
        }
        devices_url = f"{asset_library_api.url}/devices"
        asset_library_api._send_request(method="POST", url=devices_url, data=data)


In [ ]:
# import uuid
# new_uuid = uuid.uuid1()
new_uuid = "cd7a00b5-0000-0000-0000-000000000000"

def vehicle_uuid(vehicle_id):
    return f"{str(new_uuid)[0:8]}-{int(vehicle_id):04d}-{str(new_uuid)[14:]}"

for vehicle_id in gtfs_realtime_vehicle_ids:
    vehicle_dict = {
        "description": f"{agency_name} vehicle {vehicle_id}",
        "category": "device",
        "templateId": "vehiclev2",
        "state": "active",
        "deviceId": f"{agency_name}-vehicle-{vehicle_id}",
        "attributes": {
            "VID": vehicle_id,
            "name": f"vehicle-{vehicle_id}",
            "priority": "Low",
            "type": "Bus",
            "class": 10,
            "uniqueId": vehicle_uuid(vehicle_id),
        },
        "groups": {"ownedby": [f"/{agency_name}/{agency_name}"]},
        "devices": {"installedat": []},
    }
    try:
        VehicleWriter.parse_obj(vehicle_dict).create()
    except ValueError as e:
        if '{"error":"Item already exists"}' in str(e):
            # print(f"{vehicle_id} already exists, skipping")
            continue
        else:
            raise e

In [ ]:
# from gtt.service.asset_library import Vehicle

# vehicles = sorted(
#     [
#         asset_library_api.get_device(f"{agency_name}-vehicle-{vehicle_id}")
#         for vehicle_id in gtfs_realtime_vehicle_ids
#     ],
#     key=lambda v: v.VID,
# )

## IntegrationCom
Create IntegrationComs under the agency group

In [ ]:
from gtt.service.asset_library import IntegrationCom

class IntegrationComWriter(IntegrationCom):
    def create(self):
        data = {
            "attributes": {
                "serial": self.serial,
                "gttSerial": self.gtt_serial,
                "addressMAC": self.address_mac,
                "uniqueId": self.unique_id,
                "preemptionLicense": self.license,
                "integration": self.integration,
            },
            "description": self.description,
            "category": self.category,
            "templateId": self._template_id,
            "state": self.state,
            "deviceId": self.device_id,
            "groups": {
                "out": {"ownedby": [f"/{agency_name}/{agency_name}"]}
            },
            "devices": {
                "in": {"installedat": [self.vehicle_id] if self.vehicle_id else []}
            },
        }
        devices_url = f"{asset_library_api.url}/devices"
        asset_library_api._send_request(method="POST", url=devices_url, data=data)



In [ ]:
# import uuid
# new_uuid = uuid.uuid1()
new_uuid = "cd7a001c-0000-0000-0000-000000000000"

def ic_uuid(vehicle_id):
    return f"{str(new_uuid)[0:8]}-{int(vehicle_id):04d}-{str(new_uuid)[14:]}"

def ic_mac(vehicle_id):
    octet0 = f"{int(vehicle_id):04d}"[0:2]
    octet1 = f"{int(vehicle_id):04d}"[2:4]
    return f"00:00:00:d0:{octet0}:{octet1}"

for vehicle_id in gtfs_realtime_vehicle_ids:
    vehicle_dict = {
        "description": f"{agency_name} gtfs-realtime device {vehicle_id}",
        "category": "device",
        "templateId": "integrationcom",
        "state": "active",
        "deviceId": f"{agency_name}-gtfs-realtime-{vehicle_id}",
        "attributes": {
            "serial": vehicle_id,
            "gttSerial": vehicle_id,
            "addressMAC": ic_mac(vehicle_id),
            "uniqueId": ic_uuid(vehicle_id),
            "preemptionLicense": "pending",
            "integration": "gtfs-realtime",
        },
        "groups": {"ownedby": [f"/{agency_name}/{agency_name}"]},
        "devices": {"installedat": []},
    }
    try:
        IntegrationComWriter.parse_obj(vehicle_dict).create()
    except ValueError as e:
        if '{"error":"Item already exists"}' in str(e):
            # print(f"{vehicle_id} already exists, skipping")
            continue
        else:
            raise

In [ ]:
# from gtt.service.asset_library import IntegrationCom

# integration_coms = sorted(
#     [
#         asset_library_api.get_device(f"{agency_name}-gtfs-realtime-{vehicle_id}")
#         for vehicle_id in gtfs_realtime_vehicle_ids
#     ],
#     key=lambda ic: int(ic.serial),
# )

## Associate Vehicles and IntegrationComs

In [ ]:
from gtt.service.asset_library import AssetLibraryBaseModel

for vehicle_id in gtfs_realtime_vehicle_ids:
    vehicle_device_id = f"{agency_name}-vehicle-{vehicle_id}"
    integration_com_device_id = f"{agency_name}-gtfs-realtime-{vehicle_id}"
    asset_library_api._send_request(
        method="PUT",
        url=(
            f"{asset_library_api.url}/devices/{vehicle_device_id}"
            f"/installedat/devices/{integration_com_device_id}"
        )
    )

## Check Asset Library
Load vehicle/integration com assets and ensure they are associated

In [ ]:
from gtt.service.asset_library import Vehicle, IntegrationCom

vehicles = sorted(
    [
        asset_library_api.get_device(f"{agency_name}-vehicle-{vehicle_id}")
        for vehicle_id in gtfs_realtime_vehicle_ids
    ],
    key=lambda v: v.VID,
)

integration_coms = sorted(
    [
        asset_library_api.get_device(f"{agency_name}-gtfs-realtime-{vehicle_id}")
        for vehicle_id in gtfs_realtime_vehicle_ids
    ],
    key=lambda ic: int(ic.serial),
)


In [ ]:
for vehicle, integration_com in zip(vehicles, integration_coms):
    assert vehicle.integration_com == integration_com
    assert vehicle == integration_com.vehicle

In [ ]:
from gtt.service.asset_library import Agency

agency = asset_library_api.get_agency(agency_name, agency_name)
assert all(
    d in agency.devices for d in (vehicles + integration_coms)
)

## Feature Persistence

In [ ]:
agency_config

In [ ]:
import requests, json

feature_api_url = "https://2g4ct7tedk.execute-api.us-east-1.amazonaws.com/default/FeaturePersistence"

requests.post(
    feature_api_url,
    json={
        agency_name: {
            "Features": {
                "gtfs-realtime": {
                    "alerts_url": agency_config.get("alerts_url"),
                    "trip_updates_url": agency_config.get("trip_updates_url"),
                    "vehicle_positions_url": agency_config.get("vehicle_positions_url"),
                    "vehicle_id_field": agency_config.get("vehicle_id_field"),
                }
            }
        }
    },
).content

In [ ]:
requests.get(feature_api_url, params={"AgencyGUID": agency_name}).content